In [1]:
import pandas as pd 
import re
import glob
import numpy as np

# 住宅（住宅土地統計調査→df_city_house）

In [2]:
#準備（元データを整形して一旦csvにする、エクセルファイルのままだとなぜか挙動不審）
df = pd.read_excel('/Users/yuodanaka/Downloads/ojima_requests/住宅土地統計調査_住宅_築年及び構造.xlsx',header=10)
df=df.iloc[:,5:]
df=df.drop(["Unnamed: 6","Unnamed: 8","Unnamed: 10"], axis=1)
df.columns=["自治体","建物の構造","住宅の建て方","建物の階数","総数","-1970","1971-","1981","1991-","2001-","2011-","2016-"]
df.to_csv("/Users/yuodanaka/Downloads/ojima_requests/住宅土地統計調査_住宅_築年及び構造.csv",index=False)

##以下、新しい列名に揃える
#読み込み
df=pd.read_csv("/Users/yuodanaka/Downloads/ojima_requests/住宅土地統計調査_住宅_築年及び構造.csv")

#自治体コードと自治体名を切り離して、別の列をつくる
code_list=[]
for i in df["自治体"]:
    code = re.sub("\\D", "", i)
    code_list.append(code)
df["CITY_CODE"]=code_list

name_list=[]
for j in df["自治体"]:
    name=re.sub('\d+', '', j)
    name=re.sub('[_]', '', name)
    name=name.replace("\u3000", "")
    name_list.append(name)
df["CITY_NAME"]=name_list
#不要になった列を削除
df=df.drop(["自治体"],axis=1)

##建設時期を70年以前以後で分ける
#71年以後の各行の合計を出す
df=df.replace("-","0")
df_71=df.iloc[:,5:10]
df_71=df_71.astype('int')
sum_71=df_71.sum(axis=1)
#元のdfに71年以後合計の列を追加
df["SUM_1971-"]=sum_71
#不要になった列を削除
df=df.iloc[:,[0,1,2,3,4,11,12,13]]
#列名を変更
df=df.rename(columns={'SUM_1971-': '1971-'})

##市区町村単位のデータの作成
#都道府県単位のデータは不要なので、市区町村だけ取り出す
ken_list=[]#都道府県は末尾3桁が000なので、そうなっている行番号を抽出
for i,j in enumerate(df["CITY_CODE"]):
    if j[-3:]=="000":
        ken_list.append(i)
#上で抽出した行を元のデータフレームから削除
df=df.drop(ken_list)

#木造を抽出（元データにタイポがあるようなのでクエリは=ではなくstartswithで）
df_w=df.query('建物の構造.str.startswith("1_")', engine='python')
#df_w.head(10)


#木造の総数
df_w_t=df_w.query('住宅の建て方.str.startswith("0_")', engine='python')
df_w_t=df_w_t.iloc[:,3:]
df_w_t.columns=["W_TOTAL","W_-1970","CITY_CODE","CITY_NAME","W_-1971"]
#df_w_t

#木造の階別
df_w_s=df_w.query('住宅の建て方.str.startswith("3_")', engine='python')
#df_w_s.head()

#木造共同住宅で1-2階
df_w_1=df_w_s.query('建物の階数.str.startswith("04_")', engine='python')
df_w_1=df_w_1.iloc[:,3:8]
df_w_1.columns=["W_2_TOTAL","W_2_-1970","CITY_CODE","CITY_NAME","W_2_1971"]
#df_w_1.head()

#木造共同住宅で3-5階
df_w_3=df_w_s.query('建物の階数.str.startswith("05_")', engine='python')
df_w_3=df_w_3.iloc[:,3:8]
df_w_3.columns=["W_5_TOTAL","W_5_-1970","CITY_CODE","CITY_NAME","W_5_1971"]

#木造共同住宅で6階以上
df_w_6=df_w_s.query('建物の階数.str.startswith("06_")', engine='python')
df_w_6=df_w_6.iloc[:,3:8]
df_w_6.columns=["W_6_TOTAL","W_6_-1970","CITY_CODE","CITY_NAME","W_6_1971"]

#木造の結合
df_w_m=pd.merge(df_w_t,df_w_1, how="right", on=["CITY_CODE","CITY_NAME"])
df_w_m=pd.merge(df_w_m,df_w_3, how="right", on=["CITY_CODE","CITY_NAME"])
df_w_m=pd.merge(df_w_m,df_w_6, how="right", on=["CITY_CODE","CITY_NAME"])

#非木造を抽出（元データにタイポがあるようなのでクエリは=ではなくstartswithで）
df_nw=df.query('建物の構造.str.startswith("2_")', engine='python')
#df_w.head(10)


#非木造の総数
df_nw_t=df_nw.query('住宅の建て方.str.startswith("0_")', engine='python')
df_nw_t=df_nw_t.iloc[:,3:]
df_nw_t.columns=["NW_TOTAL","NW_-1970","CITY_CODE","CITY_NAME","NW_-1971"]
#df_w_t

#非木造の階別
df_nw_s=df_nw.query('住宅の建て方.str.startswith("3_")', engine='python')
#df_w_s.head()

#非木造共同住宅で1-2階
df_nw_1=df_nw_s.query('建物の階数.str.startswith("04_")', engine='python')
df_nw_1=df_nw_1.iloc[:,3:8]
df_nw_1.columns=["NW_2_TOTAL","NW_2_-1970","CITY_CODE","CITY_NAME","NW_2_1971"]
#df_w_1.head()

#非木造共同住宅で3-5階
df_nw_3=df_nw_s.query('建物の階数.str.startswith("05_")', engine='python')
df_nw_3=df_nw_3.iloc[:,3:8]
df_nw_3.columns=["NW_5_TOTAL","NW_5_-1970","CITY_CODE","CITY_NAME","NW_5_1971"]

#非木造共同住宅で6階以上
df_nw_6=df_nw_s.query('建物の階数.str.startswith("06_")', engine='python')
df_nw_6=df_nw_6.iloc[:,3:8]
df_nw_6.columns=["NW_6_TOTAL","NW_6_-1970","CITY_CODE","CITY_NAME","NW_6_1971"]

#非木造の結合
df_nw_m=pd.merge(df_nw_t,df_nw_1, how="right", on=["CITY_CODE","CITY_NAME"])
df_nw_m=pd.merge(df_nw_m,df_nw_3, how="right", on=["CITY_CODE","CITY_NAME"])
df_nw_m=pd.merge(df_nw_m,df_nw_6, how="right", on=["CITY_CODE","CITY_NAME"])

#木造と非木造の結合
df_city_house=pd.merge(df_w_m,df_nw_m,how="right", on=["CITY_CODE","CITY_NAME"])

# 公共施設（地方財政状況調査関係資料・公共施設状況調→df_kokyo）

In [3]:
#読み込み、ひとまず市区町村コードだけはstr型で
df=pd.read_excel("/Users/yuodanaka/Downloads/ojima_requests/公共施設状況調.xlsx",skiprows=[0,1,2,3,4,5,6,7,8,9,10,11],dtype={"Unnamed: 2":str})

#元データは2006年から2017年まであるので最新の17年だけ抽出
#列名はひとまずUnnamedのままで
df1=df[df['Unnamed: 0'] == 2017]

#「-」を0に置換しておく
df1=df1.replace("-",0)

#欲しいところの列名と使う列番号の辞書
#公会堂・市民会館、公民館、体育館、保健センター、青年の家・自然の家、集会施設
use_dict={"CITY_CODE":2,"CITY_NAME":4,"HALL":136,"CENTER":139,"GYM":163,"HEALTH":179,"SEINEN":181,"MEETING":188}

#欲しいところを取り出し
use_col_list=list(use_dict.values())
df2=df1.iloc[:,use_col_list]

#列名の変更
df2.columns=use_dict.keys()

#CITY_CODEを5桁にする
code_list=[]
for i in df2["CITY_CODE"]:
    i=str(i)
    i=i[:5]
    code_list.append(i)
df2["CITY_CODE"]=code_list

#都道府県計の行を削除して完成
kesu=df2[df2["CITY_NAME"]=="計"].index
df_kokyo=df2.drop(kesu)

/anaconda3/envs/odanaka/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
#CITY_NAMEからスペースを削除
name_list=[]
for name in df_kokyo["CITY_NAME"]:
    #name=re.sub('\d+', '', j)
    #name=re.sub('[_]', '', name)
    name=name.replace(" ", "")
    name_list.append(name)
df_kokyo["CITY_NAME"]=name_list

# 産業別従事者数（国勢調査→df_workers）

In [5]:
#読み込み
#これで元ファイルから整形
files = glob.glob("/Users/yuodanaka/Downloads/ojima_requests/産業/*.csv")#都道府県ごとにダウンロードできる
ken_list=[]
for file in files:
    df=pd.read_csv(file, header=12,encoding="cp932")
    city_list=[]
    for i in range(1000):
        j=i*23+23
        if j <= len(df):
            df1=df.iloc[i*23:j]
            df3=df1.iloc[:,6:]
            df3=df3.iloc[:,0:2]
            df4=df3.T
            df5=df4.rename( index={'総数（職業大分類）': df4.iloc[0,0]})
            df6=df5.drop(columns=df5.columns[[0]])
            df6.columns = df6.iloc[0]
            df6=df6.drop(index=df6.index[[0]])
            city_list.append(df6)
        else:
            pass
        df_ken=pd.concat(city_list)
    #自治体コードと自治体名を分離
    #コード
    code_list=[]
    for i in df_ken.index:
        code = re.sub("\\D", "", i)
        code_list.append(str(code))
    #code_list
    df_ken["CITY_CODE"]=code_list

    #名前
    name_list=[]
    for i in df_ken.index:
        name=re.sub('\d+', '', i)
        name_list.append(name)
    df_ken["CITY_NAME"]=name_list
    
    
    #CITY_NAMEからスペースを削除
    name_list=[]
    for name in df_ken["CITY_NAME"]:
    #name=re.sub('\d+', '', j)
    #name=re.sub('[_]', '', name)
        name=name.replace(" ", "")
        name_list.append(name)
    df_ken["CITY_NAME"]=name_list
    
    ken_list.append(df_ken)
df_worker=pd.concat(ken_list)


#インデックスを振り直す（なくても良いけどもやもやするから）
df_worker=df_worker.reset_index(drop=True)

#同じやつが三回繰り返されている。総数、男、女。ここでは総数だけ欲しいから最初だけ残して重複を削除
df_workers=df_worker.drop_duplicates(subset='CITY_CODE')

#欲しい奴は、医療・福祉、公務、運輸業、小売業等、宿泊・飲食業
use_list=[22,23]
for j, i in enumerate(df_worker.columns):
    if "医療" in i or "公務" in i or "運輸" in i or "小売" in i or "宿泊" in i:
        use_list.append(j)
    else:
        pass
df_workers=df_workers.iloc[:, use_list]

#新しいdf_workersの列名を変更
df_workers.columns=["CITY_CODE","CITY_NAME","W_TRAFFIC","W_RETAIL","W_HOTEL","W_MEDICAL","W_PUBLIC"]

#「-」を0に置換して型を変更
df_workers=df_workers.replace("-",0)
df_workers=df_workers.astype({"W_TRAFFIC":float,"W_RETAIL":float,"W_HOTEL":float,"W_MEDICAL":float,"W_PUBLIC":float})

#市区町村を抽出
ken_list=[]
for i, row in df_workers.iterrows():
    s=row["CITY_CODE"]
    if len(s) == 2:
        ken_list.append(i)
    else:
        pass
df_workers=df_workers.drop(ken_list)

#北海道の「〇〇局」を削除
name_list=df_workers["CITY_NAME"]
kyoku_list=[]
for index,row in df_workers.iterrows():
    name=row["CITY_NAME"]
    if "局" in name:
        kyoku_list.append(index)
    else:
        pass
df_workers=df_workers.drop(kyoku_list)



# 医歯薬数（医師・歯科医師・薬剤師調査→df_medical）

In [6]:
#元データを整形する関数の定義
def shaping_csv(file_pass,ddp):
    #読み込み
    df=pd.read_csv(file_pass, encoding="shift_jis", header=8,usecols=[0,1])

    #列名の変更と「-」と欠損値の処理
    df=df.dropna()
    df.columns=["CODE&CITY_NAME",ddp]
    df=df.replace("-","0")

    #CODE&CITY_NAMEを分割
    #都道府県コード、二次医療圏コード、市区町村コードを抽出
    code_list=[]
    for i in df["CODE&CITY_NAME"]:
        code = re.sub("\D", "", i)
        code_list.append(code)

    #自治体名を抽出
    name_list=[]
    for i in df["CODE&CITY_NAME"]:
        s=re.sub('\d+', '', i)
        s=s.replace('\u3000', '')
        name_list.append(s)

    #CODE&CITY_NAMEを削除し、代わりに抽出した列を追加
    df=df.drop(["CODE&CITY_NAME"],axis=1)
    df["CITY_CODE"]=code_list
    df["CITY_NAME"]=name_list

    #見やすいように並べ替え
    df=df.reindex(columns=['CITY_CODE', 'CITY_NAME', ddp])

    #市区町村だけ抽出
    tmp=pd.DataFrame()
    for index, row in df.iterrows():
        if len(row["CITY_CODE"]) ==5:
            tmp=tmp.append(row, ignore_index=True)
        else:
            pass
    
    return tmp

#整形し、一つにまとめる
#準備
file_pass_list=[
"/Users/yuodanaka/Downloads/ojima_requests/医歯薬調査_医師.csv",
"/Users/yuodanaka/Downloads/ojima_requests/医歯薬調査_歯科医師.csv",
"/Users/yuodanaka/Downloads/ojima_requests/医歯薬調査_薬剤師.csv"
]

ddp_list=["DOCTOR","DENTAL","PHARMACIST"]

#整形したものをdf_listに格納
df_list=[]
for file_pass,ddp in zip(file_pass_list,ddp_list):
    tmp1=shaping_csv(file_pass,ddp)
    df_list.append(tmp1)

#医歯薬数のデータフレーム（df_medical）の作成   
df_medical=pd.concat(df_list,axis=1)#3つを結合する
df_medical=df_medical.iloc[:,[0,1,2,5,8]]#必要な列だけ残す

# 看護師数（独自収集→df_nurse）

In [7]:
import pandas as pd
df=pd.read_csv("/Users/yuodanaka/Downloads/ojima_requests/看護師数_収集済み.csv")

#保健所を削除
df=df.drop("保健所or医療圏", axis=1)
#都道府県のみのdf_tを作成
df_t=df[df['市町村'].isnull()]
df_t=df_t.drop("市町村", axis=1)
df_t=df_t.dropna()

#市区町村のみのdf_sを作成
df_s=df[df['都道府県'].isnull()]
df_s=df_s.drop("都道府県", axis=1)
df_s=df_s.dropna()

#都道府県df_t_nurse

#df_tに都道府県コードを追加
t_code=[]
for i in range(1,48):
    s=str(i)
    s=s.zfill(2)
    t_code.append(s)
df_t["KEN_CODE"]=t_code

#df_tの列名を変更し、念のためindexを振り直す
df_t.columns=["KEN_NAME","NURSE","KEN_CODE"]
df_t_nurse=df_t.reset_index(drop=True)

#市区町村df_s_nurse

#df_sの列名を変更
df_s.columns=["CITY_NAME","NURSE"]

#市区町村コードをつける用
df_c=pd.read_csv("/Users/yuodanaka/Downloads/ojima_requests/市区町村コード_看護師数用.csv",dtype=str)

#市区町村コードと看護師数の結合
df_s_nurse=pd.merge(df_c,df_s,how="right",on="CITY_NAME")

#おそらく日本語の読み込みの問題で名前が一致しないところがあるから手動でつける
#鶴ヶ島などの「ヶ」にブレがある
#鶴ヶ島 11241
#袖ケ浦市　12229
#横浜市保土ケ谷区　14106
#14207,茅ヶ崎市
#15342,弥彦村
#20210,駒ヶ根市
#28224,南あわじ市


index_list=list(range(783,790))
code_list=["11241","12229","14106","14207","15342","20210","28224"]
for i, j in zip(index_list, code_list):
    df_s_nurse.iloc[i,0]=j

#都道府県と市区町村の結合
df_nurse=pd.merge(df_t_nurse,df_s_nurse, how="outer",on="NURSE")

#CITY_NAMEからスペースを削除
name_list=[]
for j in df_nurse["CITY_NAME"]:
    if type(j) == str:
        name=re.sub('\d+', '', j)
        name=re.sub('[_]', '', name)
        name=name.replace(" ", "")
        name_list.append(name)
    else:
        name_list.append(np.nan)
df_nurse["CITY_NAME"]=name_list

#KEN_NAMEからスペースを削除
name_list=[]
for name in df_nurse["KEN_NAME"]:
    if type(j) == str:
        name=re.sub('\d+', '', j)
        name=re.sub('[_]', '', name)
        name=name.replace(" ", "")
        name_list.append(name)
    else:
        name_list.append(np.nan)
df_nurse["KEN_NAME"]=name_list

# 総人口・世帯数、外国人数、高齢者数、高齢夫婦世帯、高齢単身（国勢調査→df_sensus）

In [8]:
#各ファイルへのパス
file0="/Users/yuodanaka/Downloads/ojima_requests/国勢調査_総人口_総世帯数.csv"#総人口
file1="/Users/yuodanaka/Downloads/ojima_requests/国勢調査_年齢別人口.csv"#年齢別人口
file2="/Users/yuodanaka/Downloads/ojima_requests/国勢調査_外国人.csv"#外国人人口
file3="/Users/yuodanaka/Downloads/ojima_requests/国勢調査_高齢世帯.csv"#高齢者単身世帯
#file4="/Users/yuodanaka/Downloads/ojima_requests/国勢調査_高齢夫婦.csv"#高齢夫婦のみ世帯

## 総人口（出力はdf_total）

#読み込み
df1=pd.read_csv(file0,skiprows=6,encoding="cp932",dtype={"地域コード":str})

#必要な列の抽出
df1=df1.iloc[:,[2,3,6,7,11,12,13]]
#列名変更、SHIKIBETSUは最後に消す、順に、総人口、総世帯数、一般（private）世帯数、施設（institution）等世帯数
df1.columns=["CITY_CODE","SHIKIBETSU","CITY_NAME","T_POP","T_HOUSE","P_HOUSE","I_HOUSE"]

#識別コードから抽出する
df1=df1.query('SHIKIBETSU == ["a", "0","1","2","3"]')
#念のためインデックスを振り直す
df1=df1.reset_index(drop=True)

#都道府県を抽出
df2=df1.query("SHIKIBETSU=='a'")
#「全国を削除」
df2=df2.drop(df2.index[[0]])
#都道府県コードをつくる
ken_list=[]
for i in df2["CITY_CODE"]:
    s=str(i)
    s=s[:2]
    ken_list.append(s)    
df2["KEN_CODE"]=ken_list
#いらないところを消す
df2=df2.drop(["CITY_CODE","SHIKIBETSU"],axis=1)
#CITYをKENに直す
df2.columns=["KEN_NAME","T_POP","T_HOUSE","P_HOUSE","I_HOUSE","KEN_CODE"]

#市区町村を抽出
df3=df1.query('SHIKIBETSU == ["0","1","2","3"]')
df3=df3.drop(["SHIKIBETSU"],axis=1)
#都道府県と市区町村を結合
df_total=pd.merge(df2,df3,how='outer',on=["T_POP","T_HOUSE","P_HOUSE","I_HOUSE"])

#「-」を0に置換
df_total=df_total.replace("-",0)
#数字に型変更
df_total=df_total.astype({'T_POP': float, 'T_HOUSE': float,'P_HOUSE': float,'I_HOUSE': float})


## 年齢別人口（出力はdf_age）

#ファイル読み込み
df5=pd.read_csv(file1,skiprows=11,encoding="cp932",dtype={"※大項目":str,"地域コード":str})
#総数、男女別で縦に入っているので、総数だけ抽出（※大項目のままではqueryで列名を読み込めないのでひとまず変更）
df5=df5.rename(columns={'※大項目': '大項目'})
df5=df5.query("大項目=='0101'")
#必要なところだけ大雑把に取り出し
df5=df5.iloc[:,2:109]
#さらに不要なところを落とす
df5=df5.drop(["境域年次(2015)","境域年次(2000)","総数（年齢）"],axis=1)

#都道府県と市区町村だけ抽出
df5=df5.query("地域識別コード ==['a','0','1','2','3']")
#全国を削除
df5=df5.drop(df5.index[[0]])
#「-」を0に置換
df5=df5.replace("-",0)

#5才以下の合計を出したいからまず5才以下だけ
df_05=df5.iloc[:,3:9]
#型をfloatに変更
df_05=df_05.astype(float)
#合計を元のdfに追加
df5["U5_POP"]=df_05.sum(axis=1)

#65才以上を抽出し合計を出し元の列に追加
df_65=df5.iloc[:,68:104]
#型をfloatに変更
df_65=df_65.astype(float)
#合計を元のdfに追加
df5["O65_POP"]=df_65.sum(axis=1)

#使うところだけ抽出
df5=df5.iloc[:,[0,1,2,104,105]]

#都道府県だけを抽出
df6=df5.query("地域識別コード =='a'")
#都道府県コードの列を追加
ken_list=[]
for i in df6["地域コード"]:
    s=i[:2]
    ken_list.append(s)   
df6["KEN_CODE"]=ken_list
#列名の変更と識別の削除
df6=df6.drop(["地域コード","地域識別コード"],axis=1)
df6.columns=["KEN_NAME","U5_POP","O65_POP","KEN_CODE"]

#市区町村の抽出
df7=df5.query("地域識別コード ==['0','1','2','3']")
#列名の変更と識別の削除
df7=df7.drop(["地域識別コード"],axis=1)
df7.columns=["CITY_CODE","CITY_NAME","U5_POP","O65_POP"]

#都道府県と市区町村の結合
df_age=pd.merge(df6,df7,how='outer',on=["U5_POP","O65_POP"])

## 外国人数（出力はdf_f）

#読み込み
df10=pd.read_csv(file2,skiprows=10,encoding="cp932",dtype={"地域コード":str})
#必要な列の抽出
df10=df10.iloc[:,[2,3,4,5]]

#「-」を0に置換して総数を数字にする
df10=df10.replace("-",0)
df10=df10.astype({"総数（国籍）":float})

#都道府県を抽出
df11=df10.query("地域識別コード == 'a'")
#全国を削除
df11=df11.drop(df11.index[[0]])
#都道府県コードの列を追加
ken_list=[]
for i in df11["地域コード"]:
    s=i[:2]
    ken_list.append(s)
df11["KEN_CODE"]=ken_list
#列名の変更と識別の削除
df11=df11.drop(["地域コード","地域識別コード"],axis=1)
df11.columns=["KEN_NAME","F_POP","KEN_CODE"]

#市区町村の抽出
df12=df10.query("地域識別コード ==['0','1','2','3']")
#列名の変更と識別の削除
df12=df12.drop(["地域識別コード"],axis=1)
df12.columns=["CITY_CODE","CITY_NAME","F_POP"]

#都道府県と市区町村の結合
df_f=pd.merge(df11,df12,how='outer',on=["F_POP"])
#「-」を0に置換してfloatに変換
df_f=df_f.replace("-",0)
df_f=df_f.astype({"F_POP":float})

## 高齢者関係（出力はdf_s）

### 高齢者単身世帯人数（df_ss）

#読み込み
df20=pd.read_csv(file3,skiprows=12,encoding="cp932",dtype={"※大項目":str,"地域コード":str})
#大項目03で世帯人員数のデータを抽出
#その前にqueryで読み込めないから列名を変更
df20=df20.rename(columns={'※大項目': '大項目'})
df20=df20.query("大項目=='03'")
#使う列だけ抽出（各種コード、自治体名、高齢者ありで世帯人員が一人）
df20=df20.iloc[:,[2,3,6,16]]

#都道府県だけ抽出
df21=df20.query("地域識別コード == 'a'")
#全国を削除
df21=df21.drop(df21.index[[0]])
#都道府県コードの追加
ken_list=[]
for i in df21["地域コード"]:
    s=i[:2]
    ken_list.append(s)    
df21["KEN_CODE"]=ken_list
#不要な列を削除し列名を変更
df21=df21.iloc[:,[2,3,4]]
df21.columns=["KEN_NAME","S_SENIOR","KEN_CODE"]

#市区町村の抽出
df22=df20.query("地域識別コード == ['0','1','2','3']")
#不要な列を削除し列名を変更
df22=df22.iloc[:,[0,2,3]]
df22.columns=["CITY_CODE","CITY_NAME","S_SENIOR"]


#都道府県と市区町村を結合
df_ss=pd.concat([df21,df22],axis=1)

#結合したものを整形する
df_ss.columns=["KEN_NAME","NS_SENIOR","KEN_CODE","CITY_CODE","CITY_NAME","S_SENIOR"]

for i in ken_list:
    tmp=df_ss.query("KEN_CODE == @i")
    j=tmp.index
    df_ss.loc[j,"S_SENIOR"]=df_ss.loc[j,"NS_SENIOR"]
    
df_ss=df_ss.drop(["NS_SENIOR"],axis=1)
df_ss=df_ss.replace("-",0)
df_ss=df_ss.astype({"S_SENIOR":float})




### 高齢者夫婦世帯数（df_sc）
#読みこむファイル名を取得（都道府県ごとのcsv）
files = glob.glob("/Users/yuodanaka/Downloads/ojima_requests/国勢調査_高齢夫婦/*.csv")#都道府県ごとにダウンロードできる

#都道府県ごとに作成するデータフレームを格納するリストを作成（都道府県、市区町村ごと）
ken_df_list=[]
shi_df_list=[]

#各ファイルごとに集計
for file in files:
    #読み込み
    df=pd.read_csv(file, skiprows=7,encoding="cp932",dtype={"地域コード":str})
    #いらない列の削除
    df=df.drop(df.columns[[0,1,4,5]],axis=1)
    
    ##都道府県ごとの集計
    df_k=df.query("地域識別コード == 'a'")
    #都道府県コードをつける
    ken_list=[]
    for i in df_k["地域コード"]:
        s=i[:2]
        ken_list.append(s)    
    df_k["KEN_CODE"]=ken_list
    #必要なところだけ抽出
    df_k=df_k.iloc[:,2:]

    #「-」を0に置換してfloatに変換
    df_k=df_k.replace("-",0)
    df_k=df_k.astype({ '総数（妻の年齢）':float, '妻が60歳未満 ':float, '妻が60～64歳':float, '妻が65～69歳':float,
           '妻が70～74歳':float, '妻が75～79歳':float, '妻が80～84歳':float, '妻が85歳以上':float})

    #都道府県ごとに抽出して必要な値を計算していく
    #ともに65才以上の夫婦
    ken_codes=list(dict.fromkeys(ken_list))#都道府県コードのリストをつくる
    sc0_dict={}#都道府県ごとに高齢者夫婦数を格納する辞書
    for code in ken_codes:
        tmp=df_k.query("KEN_CODE == @code")
        tmp=tmp.iloc[3:,4:8]
        total=sum(list(tmp.sum()))
        sc0_dict[code]=total

    #上で作成した辞書からデータフレームを作成
    df_sc0=pd.DataFrame()
    df_sc0["KEN_CODE"]=sc0_dict.keys()
    df_sc0["C_SENIOR"]=sc0_dict.values()
    
    #リストに格納
    ken_df_list.append(df_sc0)
    
    ##市区町村ごとの集計
    df_s=df.query("地域識別コード == ['0','2','3']")
    #「-」を0に置換してfloatに変換
    df_s=df_s.replace("-",0)
    df_s=df_s.astype({ '総数（妻の年齢）':float, '妻が60歳未満 ':float, '妻が60～64歳':float, '妻が65～69歳':float,
           '妻が70～74歳':float, '妻が75～79歳':float, '妻が80～84歳':float, '妻が85歳以上':float})

    #市区町村ごとに抽出して必要な値を計算していく
    #ともに65才以上の夫婦
    city_codes=list(df_s["地域コード"].unique())#都道府県コードのリストをつくる
    sc1_dict={}#市区町村ごとに高齢者夫婦数を格納する辞書
    for code in city_codes:
        tmp=df_s.query("地域コード == @code")
        tmp=tmp.iloc[3:,4:8]
        total=sum(list(tmp.sum()))
        sc1_dict[code]=total

    #上で作成した辞書からデータフレームを作成
    df_sc1=pd.DataFrame()
    df_sc1["CITY_CODE"]=sc1_dict.keys()
    df_sc1["C_SENIOR"]=sc1_dict.values()
    
    #リストに格納
    shi_df_list.append(df_sc1)

#都道府県ごと集計、市区町ごと集計をそれぞれ一つにまとめる
df_sc_ken=pd.concat(ken_df_list)
df_sc_shi=pd.concat(shi_df_list)

#都道府県と市区町村を結合
df_sc=pd.merge(df_sc_ken,df_sc_shi,how='outer',on=["C_SENIOR"])

### 単身世帯と夫婦世帯の結合

#単身高齢者と結合
df_s=pd.merge(df_ss,df_sc,how='outer',on=["KEN_CODE","CITY_CODE"])

## 全ての結合 

#3つのデータフレームを結合（2個ずつやる）
df_sensus=pd.merge(df_total,df_age,how='outer',on=["KEN_CODE","CITY_CODE","KEN_NAME","CITY_NAME"])
df_sensus=pd.merge(df_sensus,df_f,how='outer',on=["KEN_CODE","CITY_CODE","KEN_NAME","CITY_NAME"])
df_sensus=pd.merge(df_sensus,df_s,how='outer',on=["KEN_CODE","CITY_CODE","KEN_NAME","CITY_NAME"])

#CITY_NAMEからスペースを削除
tmp=df_sensus["CITY_NAME"].to_list()
name_list=[]
for name in tmp:
    if type(name)==float:
        name_list.append("")
    elif type(name)==str:
        name=name.replace(" ", "")
        name_list.append(name)

df_sensus["CITY_NAME"]=name_list

/anaconda3/envs/odanaka/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,133,134,135,136,137,138,139,140,141,142) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/odanaka/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-co

# 保健師数（地域保健健康増進事業報告→df_phn）

In [9]:
#読み込み
#保健師はpublic health nurseで長いから各単語の頭をとってphn
df_phn=pd.read_csv("/Users/yuodanaka/Downloads/ojima_requests/地域保健健康増進事業報告_保健師数.csv",skiprows=2,encoding="cp932")

#いらない行の削除といる列の抽出
df_phn=df_phn.iloc[3:50,[0,6]]

#都道府県コードを追加
ken_code=[]
for i in range(1,48):
    s=str(i)
    s=s.zfill(2)
    ken_code.append(s)
df_phn["KEN_CODE"]=ken_code

#列名の変更
df_phn.columns=["KEN_NAME","PHN","KEN_CODE"]

#KEN_NAMEからスペースを削除
name_list=[]
for name in df_phn["KEN_NAME"]:
    #name=re.sub('\d+', '', j)
    #name=re.sub('[_]', '', name)
    name=name.replace(" ", "")
    name_list.append(name)
df_phn["KEN_NAME"]=name_list

# ホテル・旅館数（経済センサス基礎調査→df_hotel）

In [10]:
#読み込み
files = glob.glob("/Users/yuodanaka/Downloads/ojima_requests/経済センサス/*.csv")
df=pd.DataFrame()
for file in files:
    tmp=pd.read_csv(file,encoding="cp932", header=8, dtype=object)
    df=df.append(tmp)

#必要な箇所だけ抽出（官民の区別は落とす、町丁目も落とす）
df=df.loc[:,['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4','75 宿泊業']]
df=df[df["Unnamed: 2"]=="総数"]
df=df.iloc[:,[1,2,3]]

#列名変更
df.columns=["CITY_CODE","CITY_NAME","HOTEL"]

#市区町村レベルの行だけを抽出
df_hotel=df.dropna(subset=["CITY_NAME"])

#「-」と「'...'」を0に置換して型変更
df_hotel=df_hotel.replace("-",0)
df_hotel=df_hotel.replace("...",0)
df_hotel=df_hotel.astype({"HOTEL":float})

#町村名に含まれている、〇〇郡を消す
name_list=df_hotel["CITY_NAME"].to_list()
new_name_list=[]
for name in name_list:
    gun=name.find("郡")
    if gun > 1 :#「郡」が含まれる市名は蒲郡市と小郡市の二つでいずれもfind（郡）は1になり、かつ、郡名が1文字のものはないため
        name=name[gun+1:]
        new_name_list.append(name)
    else:
        new_name_list.append(name)

df_hotel["CITY_NAME"]=new_name_list

#CITY_NAMEからスペースを削除
name_list=[]
for name in df_hotel["CITY_NAME"]:
    #name=re.sub('\d+', '', j)
    #name=re.sub('[_]', '', name)
    name=name.replace(" ", "")
    name_list.append(name)
df_hotel["CITY_NAME"]=name_list

In [11]:
#宮城県富谷町（04423）が2016年に富谷市（04216）に、2018年に福岡県那珂川町（40305）が那珂川市（40231	）になったのでそれを直す

#公共施設の修正
k=(df_kokyo[df_kokyo["CITY_CODE"]=="40305"]).index

df_kokyo.loc[k,"CITY_CODE"]="40231"
df_kokyo.loc[k,"CITY_NAME"]="那珂川市"

#産業別従業者数の修正
#富谷、那珂川の順
tmp=(df_workers[df_workers["CITY_CODE"]=="04423"]).index
df_workers.loc[tmp,"CITY_CODE"]="04216"
df_workers.loc[tmp,"CITY_NAME"]="富谷市"

tmp=(df_workers[df_workers["CITY_CODE"]=="40305"]).index
df_workers.loc[tmp,"CITY_CODE"]="40231"
df_workers.loc[tmp,"CITY_NAME"]="那珂川市"

#df_medicalの修正
#那珂川のみ

tmp=(df_medical[df_medical["CITY_CODE"]=="40305"]).index
df_medical.loc[tmp,"CITY_CODE"]="40231"
df_medical.loc[tmp,"CITY_NAME"]="那珂川市"

#df_nurseは関係なし

#df_sensusの修正
tmp=(df_sensus[df_sensus["CITY_CODE"]=="04423"]).index
df_sensus.loc[tmp,"CITY_CODE"]="04216"
df_sensus.loc[tmp,"CITY_NAME"]="富谷市"

tmp=(df_sensus[df_sensus["CITY_CODE"]=="40305"]).index
df_sensus.loc[tmp,"CITY_CODE"]="40231"
df_sensus.loc[tmp,"CITY_NAME"]="那珂川市"


#df_phnは関係なし

#df_hotelの修正
tmp=(df_hotel[df_hotel["CITY_CODE"]=="04423"]).index
df_hotel.loc[tmp,"CITY_CODE"]="04216"
df_hotel.loc[tmp,"CITY_NAME"]="富谷市"

tmp=(df_hotel[df_hotel["CITY_CODE"]=="40305"]).index
df_hotel.loc[tmp,"CITY_CODE"]="40231"
df_hotel.loc[tmp,"CITY_NAME"]="那珂川市"

In [12]:
#df_city_house、df_kokyo、df_workers、df_medical、df_nurse、df_sensus、df_phn、df_hotelを結合する
#一つずつmergeしていく

#国勢調査と従業者数
df_ojima=pd.merge(df_sensus,df_workers,how='outer',on="CITY_CODE")

#国勢調査と従業者数と公共施設
df_ojima1=pd.merge(df_ojima,df_kokyo,how='outer',on="CITY_CODE")

#国勢調査と従業者数と公共施設とmedical
df_ojima2=pd.merge(df_ojima1,df_medical,how='outer',on="CITY_CODE")

#国勢調査と従業者数と公共施設とmedicalとphn
df_ojima3=pd.merge(df_ojima2,df_phn,how='outer',on="KEN_CODE")

#国勢調査と従業者数と公共施設とmedicalとphnとnurse
df_ojima4=pd.merge(df_ojima3,df_nurse,how='outer',on=["KEN_CODE","CITY_CODE"])

#国勢調査と従業者数と公共施設とmedicalとphnとnurseと住宅
df_ojima5=pd.merge(df_ojima4,df_city_house,how='outer',on=["CITY_CODE"])

In [14]:
#都道府県や市区町村に関わる列を整理していく（結合した各データフレームによって微妙に違いがある）
#まず、都道府県関係はKEN_NAMEの重複を削除する（「県」などが末尾についていないのものがあってそれのせいでmergeの際にkeyにできない）
#重複列も数は47で揃っているのでdropするだけで良い
df_ojima6=df_ojima5.drop(["KEN_NAME_x","KEN_NAME_y"], axis=1)

In [24]:
ccl=[]
for i in df_ojima6.columns:
    if "CITY_NAME" in i:
        ccl.append(i)
df_ocn=df_ojima6.loc[:,ccl]

df_ocn.count()

CITY_NAME_x    1968
CITY_NAME_x    1745
CITY_NAME_x     790
CITY_NAME_y    1921
CITY_NAME_y    1900
CITY_NAME_y    1263
CITY_NAME_x    1968
CITY_NAME_x    1745
CITY_NAME_x     790
CITY_NAME_y    1921
CITY_NAME_y    1900
CITY_NAME_y    1263
CITY_NAME_x    1968
CITY_NAME_x    1745
CITY_NAME_x     790
CITY_NAME_y    1921
CITY_NAME_y    1900
CITY_NAME_y    1263
dtype: int64

In [138]:
#市区町村名は補完するように作りたい（元のデータフレームによって掲載数に差があるため）

#たくさんあると確認が大変なのでCITY_NAMEの列だけをdf_ocnとして抽出
ccl=[]
for i in df_ojima6.columns:
    if "CITY_NAME" in i:
        ccl.append(i)
df_ocn=df_ojima6.loc[:,ccl]

#全てカバーされている列を抽出し、そこから市区町村名リストを作成
df_ocn=df_ocn.iloc[:,3]
city_name_list=df_ocn.to_list()

#上で作成したリストを元のデータフレームに追加し、いらないやつを消す
ojima_requests_result=df_ojima6.drop(ccl,axis=1)
ojima_requests_result["CITY_NAME"]=city_name_list

/anaconda3/envs/odanaka/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [147]:
#都道府県コードを全部につける
ken_code_list=ojima_requests_result["KEN_CODE"].to_list()
city_code_list=ojima_requests_result["CITY_CODE"].to_list()


new_ken_code_list=[]
for kc,cc in zip(ken_code_list,city_code_list):
    if type(cc)==float:
        new_ken_code_list.append(kc)
    elif type(cc)==str:
        tmp=cc[:2]
        new_ken_code_list.append(tmp)
        
ojima_requests_result["KEN_CODE"]=new_ken_code_list

/anaconda3/envs/odanaka/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
#正しい都道府県名を追加（mergeの過程で崩れてしまっている）
#都道府県コードと都道府県名が対応しているだけのcsvを読み込み
df1=pd.read_csv("/Users/yuodanaka/Downloads/ojima_requests/都道府県コード.csv",dtype=str)

#コードと名称を辞書にする（pref_d）
tmp0=df1["コード"].to_list()
tmp1=df1["都道府県"].to_list()
pref_d={}
for i,j in zip(tmp0,tmp1):
    pref_d[i]=j

#最終アウトプットの各行の都道府県コードに対応する都道府県名をリストに格納
tmp=df["KEN_CODE"].to_list()
ken_name=[]
for i in tmp:
    for j,k in pref_d.items():
        if i==j:
            ken_name.append(k)
            break
        else:
            pass

df["KEN_NAME"]=ken_name

In [148]:
#出力
ojima_requests_result.to_csv("/Users/yuodanaka/Downloads/ojima_requests/ojima_requests_result.csv",index=False)